In [1]:
import ee
import geemap
import collections
collections.Callable = collections.abc.Callable
ee.Initialize()
import pandas as pd

In [54]:
dicto = pd.read_csv('zipIncome.csv').to_dict()
for key in dicto:
    dicto[key] = dicto[key].get(0)

### progress! just need them to actually have income!

In [82]:
Map = geemap.Map()

zips = ee.FeatureCollection("TIGER/2010/ZCTA5")
states = ee.FeatureCollection("TIGER/2016/States")

state = states.filter(ee.Filter.eq('NAME', 'California'))
ca = state.geometry()

def addCenter(feature):
    center = feature.centroid().geometry()
    insideCa = ca.contains(center, 1)
    return feature.set('containedIn', insideCa)

zipCenter = zips.map(addCenter)

# filter the feature on metadata
zipCa = zipCenter.filter(ee.Filter.eq("containedIn", True))

keys = list(dicto.keys())

zipList = zipCa.filter(ee.Filter.inList("GEOID10", keys))

In [83]:
data = ee.Dictionary(dicto)

def addIncome(feature):
    zippy = feature.get('GEOID10')
    income = data.get(zippy)
    return feature.set('medianIncome', income)

zipIncome = zipList.map(addIncome)

visParams = {'color': 'red'}
Map.addLayer(zipIncome, visParams, 'zip')
Map.centerObject(ca)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…